In [1]:
# Import dependencies
from pathlib import Path
from natsort import natsorted, index_natsorted, order_by_index
import pandas as pd
import numpy as np
import math
import re

## Read in CSV
### Order of the columns is important. Columns headers can be anything but must be present.
source plate name, source well, destination plate name, destination well, transfer volume (uL)

In [2]:
# Read in CSV file
file_path = Path('largehitpickfile.csv')
raw_hitpick_df = pd.read_csv(file_path)
raw_hitpick_df

,src_plt,src_well,dest_plt,dest_well,volume
0,src_1,1,dest_1,14,613.303333
1,src_1,1,dest_1,15,703.705556
2,src_1,1,dest_1,16,680.000000
3,src_1,1,dest_1,17,686.120000
4,src_1,1,dest_1,18,678.488889
...,...,...,...,...,...
147,src_2,1,dest_4,50,683.173333
148,src_2,1,dest_4,51,683.721111
149,src_2,1,dest_4,52,683.947778
150,src_2,1,dest_4,53,611.055556


### Rename all column headers for standardization between different hitpick worklists

In [3]:
# Source Plate Name, Source Well, Destination Plate Name, Destination Well, Transfer Volume (uL)

column1 = raw_hitpick_df.columns[0]
column2 = raw_hitpick_df.columns[1]
column3 = raw_hitpick_df.columns[2]
column4 = raw_hitpick_df.columns[3]
column5 = raw_hitpick_df.columns[4]


hitpick_df = raw_hitpick_df.rename(columns={
    column1:'Source_Plate_Name',
    column2:'Source_Well',
    column3:'Destination_Plate_Name',
    column4:'Destination_Well',
    column5:'Transfer_Volume_(uL)'
})

# Start dataframe at index 1 and rename the column to Transfer
hitpick_df.index = np.arange(1, len(hitpick_df) + 1)
hitpick_df.index.name = 'Transfer'

hitpick_df

,Source_Plate_Name,Source_Well,Destination_Plate_Name,Destination_Well,Transfer_Volume_(uL)
Transfer,,,,,
1,src_1,1,dest_1,14,613.303333
2,src_1,1,dest_1,15,703.705556
3,src_1,1,dest_1,16,680.000000
4,src_1,1,dest_1,17,686.120000
5,src_1,1,dest_1,18,678.488889
...,...,...,...,...,...
148,src_2,1,dest_4,50,683.173333
149,src_2,1,dest_4,51,683.721111
150,src_2,1,dest_4,52,683.947778


## Regular expressions to remove leading zeros from well names
Example: A01, A0100 ==> A1, A100

In [4]:
def well_number_to_identifier(number, total_well_count):
    
    # 24-well plate (4 row x 6 col)
    if total_well_count == 24:
        try:
            well_identifier = 'ABCD'[(number - 1) // 6] + str((number - 1) % 6 + 1)
            # print(f'{number} ==> {well_identifier}')
        except IndexError:
            print(f'{number} is an invalid well number. Plate does not have that well.')
    
    
    # 48-well plate (8 row x 6 col)
    if total_well_count == 48:
        try:
            well_identifier = 'ABCDEFGH'[(number - 1) // 6] + str((number - 1) % 6 + 1)
            # print(f'{number} ==> {well_identifier}')
        except IndexError:
            print(f'{number} is an invalid well number. Plate does not have that well.')
    
    
    # 96-well plate (8 row x 12 col)
    if total_well_count == 96:
        try:
            # number = 13
            # well_identifier = 'ABCDEFGH[(12 // 12)] + %02d % ((12 % 12) + 1)
            # well_identifier = B
            well_identifier = 'ABCDEFGH'[(number - 1) // 12] + str((number - 1) % 12 + 1)
            # print(f'{number} ==> {well_identifier}')
            return well_identifier
        except IndexError:
            print(f'{number} is an invalid well number. Plate does not have that well.')
                  
    # 384-well plate (16 rows x 24 col)
    if total_well_count == 384:
        try:
            well_identifier = 'ABCDEFGHIJKLMNOP'[(number - 1) // 24] + str((number - 1) % 24 + 1)
            # print(f'{number} ==> {well_identifier}')
            return well_identifier
        except IndexError:
            print(f'{number} is an invalid well number. Plate does not have that well.')

In [5]:
# Remove leading zeros and format well identifier to be uppercase
# Handle A01 if the user has that value in the column
def remove_leading_zeros(identifier):
    # Only capture the leading zeros until a nonzero digit is found
    regex = "(0+)(?=[1-9])"
    formatted_identifier = re.sub(regex, "", identifier.upper())
    print(formatted_identifier)
    
remove_leading_zeros('A01')
remove_leading_zeros('A001')
remove_leading_zeros('A01000')
remove_leading_zeros('abc01000')

A1
A1
A1000
ABC1000


In [6]:
print(str(hitpick_df.loc[hitpick_df.index[0]]))

Source_Plate_Name              src_1
Source_Well                        1
Destination_Plate_Name        dest_1
Destination_Well                  14
Transfer_Volume_(uL)      613.303333
Name: 1, dtype: object


In [7]:
# Determine if the wells are formatted numerically (1-96) or by well identifier (e.g. A1 or A01)

if (str(hitpick_df.loc[hitpick_df.index[0], 'Source_Well']).isnumeric() == True):
    
    # Replace the numerical values in hitpick_df
    hitpick_df['Source_Well'] = hitpick_df['Source_Well'].apply(lambda row: well_number_to_identifier(row, 96))
    hitpick_df['Destination_Well'] = hitpick_df['Destination_Well'].apply(lambda row: well_number_to_identifier(row, 96))

# Only check the first character to see if it is a letter
elif (str(hitpick_df.loc[hitpick_df.index[0], 'Source_Well'][0]).isalpha() == True):
    
    # Replace the well name values in hitpick_df to ensure that there are no leading zeros
    hitpick_df['Source_Well'] = hitpick_df['Source_Well'].apply(lambda row: remove_leading_zeros(row))
    hitpick_df['Destination_Well'] = hitpick_df['Destination_Well'].apply(lambda row: remove_leading_zeros(row))

In [8]:
hitpick_df

,Source_Plate_Name,Source_Well,Destination_Plate_Name,Destination_Well,Transfer_Volume_(uL)
Transfer,,,,,
1,src_1,A1,dest_1,B2,613.303333
2,src_1,A1,dest_1,B3,703.705556
3,src_1,A1,dest_1,B4,680.000000
4,src_1,A1,dest_1,B5,686.120000
5,src_1,A1,dest_1,B6,678.488889
...,...,...,...,...,...
148,src_2,A1,dest_4,E2,683.173333
149,src_2,A1,dest_4,E3,683.721111
150,src_2,A1,dest_4,E4,683.947778


In [11]:
# Get the smallest and largest transfer volumes
# Round to 1 or 2 decimal places
smallest_transfer_vol = hitpick_df['Transfer_Volume_(uL)'].min()
largest_transfer_vol = hitpick_df['Transfer_Volume_(uL)'].max()
print(f'Smallest transfer (uL): {smallest_transfer_vol}')
print(f'Largest transfer (uL): {largest_transfer_vol}')

Smallest transfer (uL): 588.5777778
Largest transfer (uL): 730.2444444


In [12]:
# Get the unique values in the source and destination plate names columns
# Add them to a total_labware list

num_source_plates = hitpick_df.Source_Plate_Name.nunique()
source_plate_names = hitpick_df.Source_Plate_Name.unique()
print(f'{num_source_plates} source plates: {source_plate_names}')

num_destination_plates = hitpick_df.Destination_Plate_Name.nunique()
destination_plate_names = hitpick_df.Destination_Plate_Name.unique()
print(f'{num_destination_plates} destination plates: {destination_plate_names}')

num_tips_required = len(hitpick_df.index - 1) # subtract 1 for header row
num_tip_boxes = math.ceil(num_tips_required / 96)
print(f'{num_tip_boxes} tip boxes needed for {num_tips_required} transfers')

8 source plates: ['src_1' 'src_2' 'src_3' 'src_4' 'src_5' 'src_6' 'src_7' 'src_8']
11 destination plates: ['dest_1' 'dest_2' 'dest_3' 'dest_4' 'dest_5' 'dest_6' 'dest_7' 'dest_8'
 'dest_9' 'dest_10' 'dest_11']
2 tip boxes needed for 152 transfers


In [ ]:
# Calculate how many decks are required to complete the hitpick...
# since there may be longer hitpicks that require more than 12 deck positions

# Deck layout
# P1    P4    P7    P10
# P2    P5    P8    P11
# P3    P6    P9    P12

total_deck_positions = 12
total_labware_positions = num_source_plates + num_destination_plates + num_tip_boxes
total_available_deck_positions = total_deck_positions - total_labware_positions

# If the total available deck positions is a negative value, that means there is too much labware
print(f'There are {total_available_deck_positions} available deck positions')

# If there are not enough deck positions then a new instrument setup is needed ...
# and the hitpick needs to be split into multiple files
additional_instrument_setups_needed = math.ceil(abs(total_available_deck_positions / 12))

if total_available_deck_positions < 0:
    print('There has to be ' + str(additional_instrument_setups_needed) + ' additional instrument setup(s)')

In [ ]:
# Add the labware TYPES (tips, source, destination) to a list and dynamically populate the deck
all_labware_names = []

for source_index in range(num_source_plates):
    all_labware_names.append(source_plate_names[source_index])
    
for dest_index in range(num_destination_plates):
    all_labware_names.append(destination_plate_names[dest_index])
    
for tips_index in range(num_tip_boxes):
    all_labware_names.append('tips_' + str(tips_index + 1))

all_labware_names

In [ ]:
# Count how many times each source labware appears in the worklist
hitpick_df.Source_Plate_Name.value_counts().sort_index(ascending=True).sort_values(ascending=False)

In [ ]:
# Count how many times each destination labware appears in the worklist
hitpick_df.Destination_Plate_Name.value_counts().sort_index(ascending=True).sort_values(ascending=False)

In [ ]:
# Count how many times each destination well appears in the worklist
hitpick_df.Destination_Well.value_counts().sort_index(ascending=True)

In [ ]:
# Count how many times each destination well appears in the worklist
# hitpick_df.Destination_Well.value_counts().sort_index(ascending=True)

# Count how many times each destination well appears in the worklist...
# and convert the series to a dataframe
destination_transfers_df = hitpick_df.Destination_Well.value_counts().sort_index(ascending=True).to_frame()

destination_transfers_df.reindex(index=natsorted(destination_transfers_df.index))

# Rename the destination well and index columns
destination_transfers_df.rename(columns={"Destination_Well": "Total_Transfers"}, inplace=True)
destination_transfers_df.index.name = 'Destination_Well'

destination_transfers_df

In [ ]:
# include another sort ==> 1. by number of transfers and 2. by plate name

In [ ]:
# If the hitpick needs to be split into multiple files, determine how to split them
# CHRONOLOGICALLY, MAXIMIZE DECK SPACE AS MUCH AS POSSIBLE
# can limit based on source, dest, or tip type

# heuristic / greedy algorithm for optimization
# treat source and dest plates the same but sort based off of transfers
# eliminate plates from least used to most used
# however may not be the most optimal with combination of source/dest

# can also do pair-wise

# can also eliminate source first

# or eliminate destination first